<a href="https://colab.research.google.com/github/vivianlu63/MotorImagery-Analysis/blob/main/Heatmap_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install nilearn --quiet
!pip install nimare --quiet
!pip install mne

     |████████████████████████████████| 4.9MB 4.3MB/s 
     |████████████████████████████████| 399kB 4.2MB/s 
     |████████████████████████████████| 2.7MB 8.3MB/s 
     |████████████████████████████████| 7.0MB 4.1MB/s 


In [3]:
import os, requests

fname = 'motor_imagery.npz'
url = "https://osf.io/ksqv8/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [4]:
from matplotlib import rcParams 
from matplotlib import pyplot as plt
import numpy as np
from matplotlib.cm import get_cmap
from scipy import signal

import mne
from mne.viz import plot_alignment, snapshot_brain_montage
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [5]:
alldat = np.load(fname, allow_pickle=True)['dat']

In [6]:
# pick subject 0 and experiment 0 (real movements)
dat1 = alldat[0][0]

# V is the voltage data
V = dat1['V'].astype('float32')

# high-pass filter above 50 Hz
b, a = signal.butter(3, [50], btype = 'high', fs=1000)
V = signal.filtfilt(b,a,V,0)

# compute smooth envelope of this signal = approx power
V = np.abs(V)**2
b, a = signal.butter(3, [10], btype = 'low', fs=1000)
V = signal.filtfilt(b,a,V,0)

# normalize each channel so its mean power is 1
V = V/V.mean(0)

In [40]:
nt, nchan = V.shape
nstim = len(dat1['t_on'])

trange = np.arange(0, 2000)
ts = dat1['t_on'][:,np.newaxis] + trange
V_epochs = np.reshape(V[ts, :], (nstim, 2000, nchan))

V_tongue = (V_epochs[dat1['stim_id']==11])
V_hand   = (V_epochs[dat1['stim_id']==12])


In [41]:
V_hand = V_hand.reshape(V_hand.shape[0] * V_hand.shape[1], V_hand.shape[2])
flipped_array = np.array([V_hand[:, sample] for sample in range(V_hand.shape[1])])

In [8]:
mne.create_info(46, dat1['srate'] * 2, 'ecog')

<Info | 7 non-empty values
 bads: []
 ch_names: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, ...
 chs: 46 ECOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 46
 projs: []
 sfreq: 2000.0 Hz
>

In [10]:
nt, nchan = V.shape
nstim = len(dat1['t_on'])

trange = np.arange(0, 2000)
ts = dat1['t_on'][:,np.newaxis] + trange
V_epochs = np.reshape(V[ts, :], (nstim, 2000, nchan))

V_tongue = (V_epochs[dat1['stim_id']==11]).mean(0)
V_hand   = (V_epochs[dat1['stim_id']==12]).mean(0)

In [11]:
epoch_length = 2000  # seconds
epochs = mne.Epochs(raw, dat1, event_id=11,
                    tmin=dat1['t_on'], tmax=dat1['t_on'] + epoch_length, baseline=None)

NameError: ignored